In [30]:
from langchain.llms.openai import OpenAI        # default model: text-davinci-003
from langchain.chat_models import ChatOpenAI    # default model: gpt-3.5-turbo

# gpt-3.5-turbo model
    # text-davinci-003에 비해 chat(대화)에 더 최적화 되어있다.
    # text-davinci-003에 비해 비용이 1/10 이다.

# InvalidRequestError: The model `text-davinci-003` has been deprecated
# 위의 Error로 인해 OpenAI 생성자에 model을 지정해주었다.
llm = OpenAI(model="gpt-3.5-turbo-instruct")
chat = ChatOpenAI()

a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

a, b

('\n\nAs of 2021, there are eight recognized planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. Some scientists also consider Pluto to be a dwarf planet.',
 'There are 8 planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')